In [1]:
# base and metaprogramming imports
from __future__ import annotations
import os
import shutil
import sys
import itertools as it
import pprint as pp
from typing import Optional
import warnings

# analysis imports
import h5py
import numpy as np
import pandas as pd
import pandas
from pandas.api.types import CategoricalDtype
import scipy
import scipy.ndimage
import skimage as ski
import ants

# visualization imports
import napari
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

# ipynb-specific imports
try:
    import ipynbname
except Exception as e:
    warnings.warn(f'`ipynbname` module import failed with error: {e!r}')
    ipynbname = None
    
if ipynbname is not None:
    _file = ipynbname.path()
else:
    # manually set the full path to the current notebook below if `ipynbname` 
    # import fails
    _file = ''
    
jobs_dir = os.path.split(_file)[0]
sources_dir = os.path.abspath(os.path.join(jobs_dir, '..'))
repo_dir = os.path.abspath(os.path.join(sources_dir, '..'))
config_dir = os.path.abspath(os.path.join(repo_dir, 'configs'))

# add sources directory to path
if sources_dir not in sys.path:
    sys.path.append(sources_dir)

# local imports
import c_swain_python_utils as csutils
from utilities import *
import imaging_dataset as imd
import prairie_view_imports as pvi

_basename = os.path.basename(_file)
_name = csutils.no_ext_basename(_file)


In [2]:
# plotting defaults
sns.set_style('ticks')

In [3]:
# logging setup
log = csutils.get_logger(_name)

debug_mode = False
default_window_level = 'debug'

log_path = os.path.join(sources_dir, '..', 'logs', f'{_basename}.log')
csutils.apply_standard_logging_config(
    file_path=log_path,
    window_level='debug' if debug_mode else default_window_level,
    window_format='debug' if debug_mode else 'default')

In [ ]:
multi_experiment_output_dir = os.path.join(
    r'G:\c_swain\zf_correlative_microscopy\analyzed_and_generated_data',
    'cs-ii-[29,33,34]_outputs')
csutils.touchdir(multi_experiment_output_dir)

activity_traces_fname = 'cs-ii-[29,33,24]_all-traces-dataframe_v1.pkl'
activity_traces_path = os.path.join(multi_experiment_output_dir, activity_traces_fname)
activity_traces_df: pd.DataFrame = csutils.load_from_disk(activity_traces_path)
log.info(f'Activity Traces Dataframe loaded...')
activity_traces_df.info()
activity_traces_df.head()

In [ ]:
stimulation_log_fname = 'full_experiment_stim_config_df.csv.pkl'
stimulation_log_path = os.path.join(multi_experiment_output_dir, stimulation_log_fname)
stimulation_log_df: pd.DataFrame = csutils.load_from_disk(stimulation_log_path)
log.info(f'Stimulation Log Dataframe loaded...')
stimulation_log_df.info()
stimulation_log_df.head()

In [ ]:
swim_bout_stat_fname = 'cs-ii-[29,33,34]_swim-bout-statistics.pd.pkl'
swim_bout_stat_path = os.path.join(
    multi_experiment_output_dir, swim_bout_stat_fname)
swim_bout_stat_df: pd.DataFrame = (
    csutils.load_from_disk(swim_bout_stat_path))
log.info(f'Swim Bout Statistics Dataframe loaded...')
swim_bout_stat_df.info()
swim_bout_stat_df.head()

In [4]:
tail_movement_timecourse_fname = 'cs-ii-[29,33,34]_tail-movement-timecourses.pd.pkl'
tail_movement_timecourse_path = os.path.join(
    multi_experiment_output_dir, tail_movement_timecourse_fname)
tail_movement_timecourse_df: pd.DataFrame = (
    csutils.load_from_disk(tail_movement_timecourse_path))
log.info(f'Tail Movement Timecourse Dataframe loaded...')
tail_movement_timecourse_df.info()
tail_movement_timecourse_df.head()

local-nb_cs-ii-[33,34]_activity_stimulus_overlays (  10): INFO : Activity Traces Dataframe loaded...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7428722 entries, 0 to 7428721
Data columns (total 16 columns):
 #   Column                    Dtype   
---  ------                    -----   
 0   region                    category
 1   super-region              category
 2   lateral-side              category
 3   time                      float64 
 4   fluorescence-quantile_95  float64 
 5   fluorescence-quantile_75  float64 
 6   fluorescence-quantile_50  float64 
 7   fluorescence-quantile_25  float64 
 8   fluorescence-quantile_05  float64 
 9   fluorescence-mean         float64 
 10  fluorescence-variance     float64 
 11  dorsoventral-position     float64 
 12  lateral-position          float64 
 13  anteroposterior-position  float64 
 14  experiment-id             category
 15  run-id                    category
dtypes: category(5), float64(11)
memory usage: 666.0 MB
local-nb_c

In [11]:
stimulation_log_df

,description,duration_s,left,middle,right,flag,stim-type,actual-rel-start-time_s,experiment-id,run-id,stim-step-index
0,break-1,300.0,NaN,NaN,NaN,False,equilibration,0.000000,cs-ii-29,395,0
1,pulse-1,60.0,NaN,fish water,NaN,False,tactile-pulse,299.999816,cs-ii-29,395,1
2,break-2,120.0,NaN,NaN,NaN,False,break,360.000666,cs-ii-29,395,2
3,pulse-2,60.0,NaN,fish water,NaN,False,tactile-pulse,480.001961,cs-ii-29,395,3
4,break-3,120.0,NaN,NaN,NaN,False,break,540.002058,cs-ii-29,395,4
...,...,...,...,...,...,...,...,...,...,...,...
245,wash,120.0,fish water,fish water,fish water,False,break,848.006102,cs-ii-34,209,19
246,aa sustain,30.0,amino acid (1.3 mM),fish water,amino acid (1.3 mM),False,aa-1.3mM,968.005482,cs-ii-34,209,20
247,wash,120.0,fish water,fish water,fish water,False,break,998.005913,cs-ii-34,209,21
248,cadaverine sustain,30.0,cadaverine (2 mM),fish water,cadaverine (2 mM),False,cdv-2mM,1118.006394,cs-ii-34,209,22


In [40]:
# COLUMN NAMES (CN)
REGION_CN = 'region'
SUPER_REGION_CN = 'super-region'
LAT_SIDE_CN = 'lateral-side'
TIME_CN = 'time'
Q95_CN = 'fluorescence-quantile_95'
Q75_CN = 'fluorescence-quantile_75'
Q50_CN = 'fluorescence-quantile_50'
Q25_CN = 'fluorescence-quantile_25'
Q05_CN = 'fluorescence-quantile_05'
MEAN_CN = 'fluorescence-mean'       
VAR_CN = 'fluorescence-variance'   
DV_POS_CN = 'dorsoventral-position'   
LAT_POS_CN = 'lateral-position'        
AP_POS_CN = 'anteroposterior-position'
EXPERIMENT_ID_CN = 'experiment-id'           
RUN_ID_CN = 'run-id'                  
DESCRIPTION_CN = 'description'             
DURATION_CN = 'duration_s'              
MANIFOLD_LEFT_CN = 'left'                   
MANIFOLD_MIDDLE_CN = 'middle'                  
MANIFOLD_RIGHT_CN = 'right'                   
FLAG_CN = 'flag'                    
STIM_TYPE_CN = 'stim-type'               
REL_START_TIME_CN = 'actual-rel-start-time_s'                 
STIM_STEP_INDEX_CN = 'stim-step-index'         

In [14]:
unique_stim_types = stimulation_log_df[STIM_TYPE_CN].unique()
unique_stim_types

['equilibration', 'tactile-pulse', 'break', 'cdv-2mM', 'aa-1.3mM', 'aa-1.3mM&cdv-2mM', 'control', 'rest-baseline', 'cdv-2mM&aa-1.3mM']
Categories (9, object): ['aa-1.3mM', 'aa-1.3mM&cdv-2mM', 'break', 'cdv-2mM', ..., 'control', 'equilibration', 'rest-baseline', 'tactile-pulse']

In [76]:
def chunks(seq, size):
    _it = iter(seq)
    while True:
        ret = tuple(next(_it) for _ in range(size))
        if len(ret) == size:
            yield ret
        else:
            raise StopIteration()

In [77]:
all_regions = activity_traces_df[REGION_CN].unique()

reg = all_regions[0]

loc_arr = np.zeros((len(all_regions),))
for i, reg in enumerate(all_regions):
    res = (activity_traces_df
           .loc[activity_traces_df[REGION_CN] == reg, :]
           .reset_index()
           .loc[0, AP_POS_CN])
    loc_arr[i] = res
    
all_regions = all_regions[np.argsort(loc_arr)]
all_regions, loc_arr[np.argsort(loc_arr)]

(['Lateral Line Neuromast N, Ganglia', 'Olfactory Epithelium, Ganglia', 'Olfactory Bulb, Telencephalon', 'Olfactory bulb dopaminergic neuron areas, Tel..., 'Olig2 Cluster, Telencephalon', ..., 'Isl1 stripe - motorneurons, Spinal Cord', 'Dorsal Sparse Isl1 cluster, Spinal Cord', 'Gad1b Stripe 2, Spinal Cord', 'Spinal Cord', 'Vmat2 Stripe2, Spinal Cord']
 Length: 293
 Categories (293, object): ['6.7FDhcrtR-Gal4 Cluster 1, Rhombencephalon', '6.7FDhcrtR-Gal4 Cluster 2 Sparse, Rhombenceph..., '6.7FDhcrtR-Gal4 Cluster 3, Rhombencephalon', '6.7FDhcrtR-Gal4 Cluster 4, Rhombencephalon', ..., 'Vmat2 cluster of paraventricular organ, Mesen..., 'Vmat2 cluster, Telencephalon', 'Vmat2 cluster2, Mesencephalon', 'X Vagus motorneuron cluster, Rhombencephalon'],
 array([0.06344404, 0.08408829, 0.10214415, 0.10937681, 0.12268165,
        0.13710157, 0.13935534, 0.14849849, 0.14956865, 0.1571719 ,
        0.15886315, 0.16027254, 0.16265537, 0.16408585, 0.16958026,
        0.17162645, 0.17439997, 0.1853449

## Generating Plots for Neural Responses to Tactile Pulses

In [15]:
selected_stim_type = 'tactile-pulse'
stim_log_subset_df: pd.DataFrame = stimulation_log_df.loc[stimulation_log_df[STIM_TYPE_CN] == selected_stim_type, :]
stim_log_subset_df.info()
stim_log_subset_df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12 entries, 1 to 26
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   description              12 non-null     object  
 1   duration_s               12 non-null     float64 
 2   left                     0 non-null      category
 3   middle                   12 non-null     category
 4   right                    0 non-null      category
 5   flag                     12 non-null     bool    
 6   stim-type                12 non-null     category
 7   actual-rel-start-time_s  12 non-null     float64 
 8   experiment-id            12 non-null     category
 9   run-id                   12 non-null     category
 10  stim-step-index          12 non-null     int64   
dtypes: bool(1), category(6), float64(2), int64(1), object(1)
memory usage: 2.3+ KB


,description,duration_s,left,middle,right,flag,stim-type,actual-rel-start-time_s,experiment-id,run-id,stim-step-index
1,pulse-1,60.0,NaN,fish water,NaN,False,tactile-pulse,299.999816,cs-ii-29,395,1
3,pulse-2,60.0,NaN,fish water,NaN,False,tactile-pulse,480.001961,cs-ii-29,395,3
5,pulse-3,60.0,NaN,fish water,NaN,False,tactile-pulse,660.002661,cs-ii-29,395,5
8,pulse-1,60.0,NaN,fish water,NaN,False,tactile-pulse,299.999124,cs-ii-29,396,1
10,pulse-2,60.0,NaN,fish water,NaN,False,tactile-pulse,479.998784,cs-ii-29,396,3
12,pulse-3,60.0,NaN,fish water,NaN,False,tactile-pulse,659.999918,cs-ii-29,396,5
15,pulse-1,60.0,NaN,fish water,NaN,False,tactile-pulse,299.999802,cs-ii-29,398,1
17,pulse-2,60.0,NaN,fish water,NaN,False,tactile-pulse,480.000258,cs-ii-29,398,3
19,pulse-3,60.0,NaN,fish water,NaN,False,tactile-pulse,659.999595,cs-ii-29,398,5
22,pulse-1,60.0,NaN,fish water,NaN,False,tactile-pulse,300.001012,cs-ii-29,399,1


In [17]:
unique_stim_durations = stim_log_subset_df.loc[:, DURATION_CN].unique()
unique_stim_durations

array([60.])

In [90]:
%matplotlib qt
stim_duration = unique_stim_durations[0]
time_pre_stim_start = np.min([stim_duration / 2, 20])
time_post_stim = np.min([stim_duration * 2, 90])
time_post_stim_start = stim_duration + time_post_stim

sampling_period = 0.5
ref_time_array = np.arange(-1 * time_pre_stim_start + 0.5, time_post_stim_start - 0.5, 
                           sampling_period)
ref_time_index = pd.Index(ref_time_array, name=TIME_CN)
log.info(f'{ref_time_array.shape = }')
log.info(f'{ref_time_index = }')

stim_seq_dfs = []
for _, stim_loc_row in stim_log_subset_df.iterrows():
    log.info(f'{stim_loc_row[EXPERIMENT_ID_CN] = }, {stim_loc_row[RUN_ID_CN] = }')
    
    activity_traces_subset_df: pd.DataFrame = activity_traces_df.loc[
        (activity_traces_df[EXPERIMENT_ID_CN] == stim_loc_row[EXPERIMENT_ID_CN]) &
        (activity_traces_df[RUN_ID_CN] == stim_loc_row[RUN_ID_CN]), :].copy()
    activity_traces_subset_df[TIME_CN] = activity_traces_subset_df[TIME_CN] - stim_loc_row[REL_START_TIME_CN]
    activity_traces_subset_df = activity_traces_subset_df.loc[
        (activity_traces_subset_df[TIME_CN] >= (-1 * time_pre_stim_start)) &
        (activity_traces_subset_df[TIME_CN] <= time_post_stim_start), :]
    
    log.info(f'{activity_traces_subset_df.shape = }')
    
    activity_traces_subset_data = activity_traces_subset_df.pivot(
        index=TIME_CN,
        columns=[REGION_CN, LAT_SIDE_CN, EXPERIMENT_ID_CN, RUN_ID_CN],
        values=MEAN_CN)
    log.info(f'{activity_traces_subset_data.shape = }')
    
    baseline_mean = activity_traces_subset_data.loc[
        (activity_traces_subset_data.index < -1), :].quantile(0.5, axis=0)
    log.info(f'{baseline_mean.shape = }')
    
    activity_traces_subset_data = (activity_traces_subset_data - baseline_mean) / baseline_mean
    interp_res = scipy.interpolate.interp1d(
        activity_traces_subset_data.index,
        activity_traces_subset_data,
        axis=0,
        kind='linear',
        bounds_error=False)(ref_time_array)
    interp_res = scipy.ndimage.gaussian_filter1d(
        interp_res, 0.5 / sampling_period, axis=0, mode='nearest')
    
    log.info(f'{interp_res.shape = }')
    
    activity_traces_subset_data = pd.DataFrame(
        interp_res,
        index=ref_time_index,
        columns=activity_traces_subset_data.columns)            
    
    # make filtered data back into dataframe
    filt_activity_traces_subset_df = (
        activity_traces_subset_data.melt(value_name=MEAN_CN, ignore_index=False)
        .reset_index())
    log.info(f'{filt_activity_traces_subset_df.shape = }')
    
    stim_seq_dfs.append(filt_activity_traces_subset_df)
    
stim_seq_df = pd.concat(stim_seq_dfs, axis=0, ignore_index=True)

stim_seq_df.info()
stim_seq_df

matplotlib.pyplot ( 339): DEBUG : Loaded backend Qt5Agg version 5.15.7.
local-nb_cs-ii-[33,34]_activity_stimulus_overlays (  11): INFO : ref_time_array.shape = (338,)
local-nb_cs-ii-[33,34]_activity_stimulus_overlays (  12): INFO : ref_time_index = Float64Index([-19.5, -19.0, -18.5, -18.0, -17.5, -17.0, -16.5, -16.0, -15.5,
              -15.0,
              ...
              144.5, 145.0, 145.5, 146.0, 146.5, 147.0, 147.5, 148.0, 148.5,
              149.0],
             dtype='float64', name='time', length=338)
local-nb_cs-ii-[33,34]_activity_stimulus_overlays (  16): INFO : stim_loc_row[EXPERIMENT_ID_CN] = 'cs-ii-29', stim_loc_row[RUN_ID_CN] = 395
local-nb_cs-ii-[33,34]_activity_stimulus_overlays (  26): INFO : activity_traces_subset_df.shape = (100206, 16)
local-nb_cs-ii-[33,34]_activity_stimulus_overlays (  32): INFO : activity_traces_subset_data.shape = (171, 586)
local-nb_cs-ii-[33,34]_activity_stimulus_overlays (  36): INFO : baseline_mean.shape = (586,)
local-nb_cs-ii-[33,34]_

,time,region,lateral-side,experiment-id,run-id,fluorescence-mean
0,-19.5,"Mauthner Cell Axon Cap, Rhombencephalon",left,cs-ii-29,395,-0.401767
1,-19.0,"Mauthner Cell Axon Cap, Rhombencephalon",left,cs-ii-29,395,-0.485428
2,-18.5,"Mauthner Cell Axon Cap, Rhombencephalon",left,cs-ii-29,395,-0.433889
3,-18.0,"Mauthner Cell Axon Cap, Rhombencephalon",left,cs-ii-29,395,-0.204859
4,-17.5,"Mauthner Cell Axon Cap, Rhombencephalon",left,cs-ii-29,395,-0.003334
...,...,...,...,...,...,...
2376811,147.0,Rhombencephalon,right,cs-ii-29,399,-0.004153
2376812,147.5,Rhombencephalon,right,cs-ii-29,399,-0.003659
2376813,148.0,Rhombencephalon,right,cs-ii-29,399,-0.006811
2376814,148.5,Rhombencephalon,right,cs-ii-29,399,-0.009070


In [ ]:
%matplotlib qt

for i, region_chunk in enumerate(chunks(all_regions, 16)):
    # log.info(f'{region_chunk = }')
    
    subset_df = stim_seq_df.loc[stim_seq_df[REGION_CN].isin(region_chunk), :]
    
    fig = sns.relplot(
        data=subset_df,
        x=TIME_CN,
        y=MEAN_CN,
        hue=LAT_SIDE_CN,
        col=REGION_CN,
        col_wrap=4,
        kind='line',
        facet_kws={'sharey': True, 'sharex': True},
        errorbar=('se', 2),
        height=2.5,
        aspect=1.5)

    (fig
     .map(plt.axvline, x=0, color='k', dashes=(2, 1), linewidth=1, zorder=0)
     .map(plt.axhline, y=0, color='k', dashes=(2, 1), linewidth=1, zorder=0)
     .map(plt.fill_between, x=[0, stim_duration], y1=[-5, -5], y2=[5, 5], 
          color='k', alpha=0.1, linewidth=0, zorder=-1)
     .set_axis_labels('Time (s)', 'Delta F / F (a.u.)')
     .set_titles('{col_name}')
     .set(xlim=(-1 * time_pre_stim_start, time_post_stim_start),
          ylim=(-0.2, 0.4))
     .tight_layout(h_pad=0.5, w_pad=0, rect=(0, 0, 0.95, 0.98)))
    
    fig.fig.suptitle(f'Aggregated Results for Stimulation: '
                     f'{selected_stim_type} - {stim_duration} s Pulse '
                     f'(two standard errors shaded)', 
                     y=0.99)
    
    # sns.despine(offset=2, trim=True)

    save_path = os.path.join(multi_experiment_output_dir, 
                             'figures',
                             f'stimulation-{selected_stim_type}',
                             f'{selected_stim_type}_'
                             f'{stim_duration:.0f}s-pulse_'
                             f'{i:03d}-region-set_'
                             f'aggregated-results.png')
    csutils.touchdir(os.path.dirname(save_path))
    log.info(f'Saving figure to disk...')
    fig.savefig(save_path, dpi=300)

## Generating Plots for Neural Responses to Control, AA, and Cadaverine Stimulation

In [95]:
selected_stim_types = ['control', 'cdv-2mM', 'aa-1.3mM']
stim_log_subset_df: pd.DataFrame = (
    stimulation_log_df
    .loc[stimulation_log_df[STIM_TYPE_CN].isin(selected_stim_types), :])
stim_log_subset_df.info()
stim_log_subset_df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 84 entries, 29 to 248
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   description              84 non-null     object  
 1   duration_s               84 non-null     float64 
 2   left                     84 non-null     category
 3   middle                   84 non-null     category
 4   right                    84 non-null     category
 5   flag                     84 non-null     bool    
 6   stim-type                84 non-null     category
 7   actual-rel-start-time_s  84 non-null     float64 
 8   experiment-id            84 non-null     category
 9   run-id                   84 non-null     category
 10  stim-step-index          84 non-null     int64   
dtypes: bool(1), category(6), float64(2), int64(1), object(1)
memory usage: 5.6+ KB


,description,duration_s,left,middle,right,flag,stim-type,actual-rel-start-time_s,experiment-id,run-id,stim-step-index
29,cadaverine pulse,5.0,cadaverine (2 mM),fish water,cadaverine (2 mM),False,cdv-2mM,299.999359,cs-ii-33,188,1
31,cadaverine pulse,5.0,cadaverine (2 mM),fish water,cadaverine (2 mM),False,cdv-2mM,335.000642,cs-ii-33,188,3
33,aa pulse,5.0,amino acid (1.3 mM),fish water,amino acid (1.3 mM),False,aa-1.3mM,370.001296,cs-ii-33,188,5
35,aa pulse,5.0,amino acid (1.3 mM),fish water,amino acid (1.3 mM),False,aa-1.3mM,405.002275,cs-ii-33,188,7
44,cadaverine sustain,30.0,cadaverine (2 mM),fish water,cadaverine (2 mM),False,cdv-2mM,668.007909,cs-ii-33,188,16
...,...,...,...,...,...,...,...,...,...,...,...
233,aa pulse,5.0,amino acid (1.3 mM),fish water,amino acid (1.3 mM),False,aa-1.3mM,405.001775,cs-ii-34,209,7
242,cadaverine sustain,30.0,cadaverine (2 mM),fish water,cadaverine (2 mM),False,cdv-2mM,668.004826,cs-ii-34,209,16
244,aa sustain,30.0,amino acid (1.3 mM),fish water,amino acid (1.3 mM),False,aa-1.3mM,818.005670,cs-ii-34,209,18
246,aa sustain,30.0,amino acid (1.3 mM),fish water,amino acid (1.3 mM),False,aa-1.3mM,968.005482,cs-ii-34,209,20


In [98]:
unique_stim_durations_dict = dict()
for stim_type in selected_stim_types:
    unique_stim_durations_dict[stim_type] = (
        stim_log_subset_df
        .loc[stim_log_subset_df[STIM_TYPE_CN] == stim_type, DURATION_CN]
        .unique())

duration_selections = unique_stim_durations_dict['control']

unique_stim_durations_dict

{'control': array([10., 40.]),
 'cdv-2mM': array([ 5., 30., 10., 40.]),
 'aa-1.3mM': array([ 5., 30., 10., 40.])}

In [99]:
stim_log_subset_df = stim_log_subset_df.loc[
    stim_log_subset_df[DURATION_CN].isin(duration_selections), :]
stim_log_subset_df.info()
stim_log_subset_df

<class 'pandas.core.frame.DataFrame'>
Int64Index: 36 entries, 125 to 200
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   description              36 non-null     object  
 1   duration_s               36 non-null     float64 
 2   left                     36 non-null     category
 3   middle                   36 non-null     category
 4   right                    36 non-null     category
 5   flag                     36 non-null     bool    
 6   stim-type                36 non-null     category
 7   actual-rel-start-time_s  36 non-null     float64 
 8   experiment-id            36 non-null     category
 9   run-id                   36 non-null     category
 10  stim-step-index          36 non-null     int64   
dtypes: bool(1), category(6), float64(2), int64(1), object(1)
memory usage: 3.4+ KB


,description,duration_s,left,middle,right,flag,stim-type,actual-rel-start-time_s,experiment-id,run-id,stim-step-index
125,control-short-left,10.0,negative control,fish water,fish water,False,control,179.998946,cs-ii-34,202,1
127,control-short-right,10.0,fish water,fish water,negative control,False,control,250.000247,cs-ii-34,202,3
129,control-short-both,10.0,negative control,fish water,negative control,False,control,320.000146,cs-ii-34,202,5
131,control-long-left,40.0,negative control,fish water,fish water,False,control,390.000474,cs-ii-34,202,7
133,control-long-right,40.0,fish water,fish water,negative control,False,control,520.000670,cs-ii-34,202,9
135,control-long-both,40.0,negative control,fish water,negative control,False,control,650.001760,cs-ii-34,202,11
138,control-short-right,10.0,fish water,fish water,negative control,False,control,179.998987,cs-ii-34,203,1
140,control-long-left,40.0,negative control,fish water,fish water,False,control,250.000507,cs-ii-34,203,3
142,control-long-both,40.0,negative control,fish water,negative control,False,control,380.001330,cs-ii-34,203,5
144,control-short-both,10.0,negative control,fish water,negative control,False,control,510.001744,cs-ii-34,203,7


In [106]:
SPATIAL_LAYOUT_CN = 'stimulus-direction'
stim_log_subset_df[SPATIAL_LAYOUT_CN] = (
    (stim_log_subset_df[MANIFOLD_LEFT_CN] != 'fish water') * 1
    + (stim_log_subset_df[MANIFOLD_RIGHT_CN] != 'fish water') * 2)
stim_log_subset_df[SPATIAL_LAYOUT_CN] = (
    stim_log_subset_df[SPATIAL_LAYOUT_CN]
    .astype(CategoricalDtype(categories=[1, 2, 3]))
    .cat.rename_categories(['left', 'right', 'both']))
stim_log_subset_df

,description,duration_s,left,middle,right,flag,stim-type,actual-rel-start-time_s,experiment-id,run-id,stim-step-index,stimulus-direction
125,control-short-left,10.0,negative control,fish water,fish water,False,control,179.998946,cs-ii-34,202,1,left
127,control-short-right,10.0,fish water,fish water,negative control,False,control,250.000247,cs-ii-34,202,3,right
129,control-short-both,10.0,negative control,fish water,negative control,False,control,320.000146,cs-ii-34,202,5,both
131,control-long-left,40.0,negative control,fish water,fish water,False,control,390.000474,cs-ii-34,202,7,left
133,control-long-right,40.0,fish water,fish water,negative control,False,control,520.000670,cs-ii-34,202,9,right
135,control-long-both,40.0,negative control,fish water,negative control,False,control,650.001760,cs-ii-34,202,11,both
138,control-short-right,10.0,fish water,fish water,negative control,False,control,179.998987,cs-ii-34,203,1,right
140,control-long-left,40.0,negative control,fish water,fish water,False,control,250.000507,cs-ii-34,203,3,left
142,control-long-both,40.0,negative control,fish water,negative control,False,control,380.001330,cs-ii-34,203,5,both
144,control-short-both,10.0,negative control,fish water,negative control,False,control,510.001744,cs-ii-34,203,7,both


In [107]:
stim_log_subset_short_df = stim_log_subset_df.loc[
    stim_log_subset_df[DURATION_CN] <= 10, :]
stim_log_subset_long_df = stim_log_subset_df.loc[
    stim_log_subset_df[DURATION_CN] > 10, :]

stim_log_subset_short_df.info()
stim_log_subset_long_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 125 to 194
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   description              18 non-null     object  
 1   duration_s               18 non-null     float64 
 2   left                     18 non-null     category
 3   middle                   18 non-null     category
 4   right                    18 non-null     category
 5   flag                     18 non-null     bool    
 6   stim-type                18 non-null     category
 7   actual-rel-start-time_s  18 non-null     float64 
 8   experiment-id            18 non-null     category
 9   run-id                   18 non-null     category
 10  stim-step-index          18 non-null     int64   
 11  stimulus-direction       18 non-null     category
dtypes: bool(1), category(7), float64(2), int64(1), object(1)
memory usage: 2.7+ KB
<class 'pandas.core.frame.DataFrame'>
In

In [ ]:
%matplotlib qt
stim_duration = 10
time_pre_stim_start = 20
time_post_stim = 50
time_post_stim_start = stim_duration + time_post_stim

sampling_period = 0.5
ref_time_array = np.arange(-1 * time_pre_stim_start, time_post_stim_start, 
                           sampling_period)
ref_time_index = pd.Index(ref_time_array, name=TIME_CN)
log.info(f'{ref_time_array.shape = }')
log.info(f'{ref_time_index = }')

stim_seq_dfs = []
for _, stim_loc_row in stim_log_subset_short_df.iterrows():
    log.info(f'{stim_loc_row[EXPERIMENT_ID_CN] = }, {stim_loc_row[RUN_ID_CN] = }')
    
    activity_traces_subset_df: pd.DataFrame = activity_traces_df.loc[
        (activity_traces_df[EXPERIMENT_ID_CN] == stim_loc_row[EXPERIMENT_ID_CN]) &
        (activity_traces_df[RUN_ID_CN] == stim_loc_row[RUN_ID_CN]), :].copy()
    
    if activity_traces_subset_df.shape[0] == 0:
        continue
    
    activity_traces_subset_df[TIME_CN] = activity_traces_subset_df[TIME_CN] - stim_loc_row[REL_START_TIME_CN]
    activity_traces_subset_df = activity_traces_subset_df.loc[
        (activity_traces_subset_df[TIME_CN] >= (-1 * (2 + time_pre_stim_start))) &
        (activity_traces_subset_df[TIME_CN] <= (2 + time_post_stim_start)), :]
    
    log.info(f'{activity_traces_subset_df.shape = }')
    
    activity_traces_subset_data = activity_traces_subset_df.pivot(
        index=TIME_CN,
        columns=[REGION_CN, LAT_SIDE_CN, EXPERIMENT_ID_CN, RUN_ID_CN],
        values=MEAN_CN)
    log.info(f'{activity_traces_subset_data.shape = }')    
    
    baseline_mean = activity_traces_subset_data.loc[
        (activity_traces_subset_data.index < -1), :].quantile(0.5, axis=0)
    log.info(f'{baseline_mean.shape = }')
    
    activity_traces_subset_data = (activity_traces_subset_data - baseline_mean) / baseline_mean
    interp_res = scipy.interpolate.interp1d(
        activity_traces_subset_data.index,
        activity_traces_subset_data,
        axis=0,
        kind='linear',
        bounds_error=False)(ref_time_array)
    interp_res = scipy.ndimage.gaussian_filter1d(
        interp_res, 0.5 / sampling_period, axis=0, mode='nearest')
    
    log.info(f'{interp_res.shape = }')
    
    activity_traces_subset_data = pd.DataFrame(
        interp_res,
        index=ref_time_index,
        columns=activity_traces_subset_data.columns)       
    
    # make filtered data back into dataframe
    filt_activity_traces_subset_df = (
        activity_traces_subset_data.melt(value_name=MEAN_CN, ignore_index=False)
        .reset_index())
    filt_activity_traces_subset_df[SPATIAL_LAYOUT_CN] = stim_loc_row[SPATIAL_LAYOUT_CN]
    filt_activity_traces_subset_df[STIM_TYPE_CN] = stim_loc_row[STIM_TYPE_CN]
    log.info(f'{filt_activity_traces_subset_df.shape = }')
    
    stim_seq_dfs.append(filt_activity_traces_subset_df)
    
stim_seq_df = pd.concat(stim_seq_dfs, axis=0, ignore_index=True)

stim_seq_df.info()
stim_seq_df

In [ ]:
%matplotlib qt

for i, region_chunk in enumerate(chunks(all_regions, 1)):
    # log.info(f'{region_chunk = }')
    
    save_path = os.path.join(multi_experiment_output_dir, 
                             'figures',
                             f'stimulation-multi-stim-type',
                             f'multi_stim_'
                             f'{stim_duration:.0f}s-pulse_'
                             f'{i:03d}-region_'
                             f'aggregated-results.png')
    
    if os.path.exists(save_path):
        continue
                
    try:
        subset_df = stim_seq_df.loc[stim_seq_df[REGION_CN].isin(region_chunk), :]
        
        fig = sns.relplot(
            data=subset_df,
            x=TIME_CN,
            y=MEAN_CN,
            hue=LAT_SIDE_CN,
            col=SPATIAL_LAYOUT_CN,
            row=STIM_TYPE_CN,        
            kind='line',
            facet_kws={'sharey': True, 'sharex': True},
            errorbar=('se', 2),
            height=2.5,
            aspect=1.5)
    
        (fig
         .map(plt.axvline, x=0, color='k', dashes=(2, 1), linewidth=1, zorder=0)
         .map(plt.axhline, y=0, color='k', dashes=(2, 1), linewidth=1, zorder=0)
         .map(plt.fill_between, x=[0, stim_duration], y1=[-5, -5], y2=[5, 5], 
              color='k', alpha=0.1, linewidth=0, zorder=-1)
         .set_axis_labels('Time (s)', 'Delta F / F (a.u.)')
         .set(xlim=(-1 * time_pre_stim_start, time_post_stim_start),
              ylim=(-0.3, 0.6))
         .tight_layout(h_pad=0.75, w_pad=0.5, rect=(0, 0, 0.93, 0.93)))
        
        fig.fig.suptitle(f'{region_chunk[0]}'
                         f'\nAggregated Results for Stimulation with different Odors, '
                         f'{stim_duration} s Pulse '
                         f'(two standard errors shaded)', 
                         y=0.99)
        
        # sns.despine(offset=2, trim=True)
    
        save_path = os.path.join(multi_experiment_output_dir, 
                                 'figures',
                                 f'stimulation-multi-stim-type',
                                 f'multi_stim_'
                                 f'{stim_duration:.0f}s-pulse_'
                                 f'{i:03d}-region_'
                                 f'aggregated-results.png')
        csutils.touchdir(os.path.dirname(save_path))
        log.info(f'Saving figure to disk...')
        fig.savefig(save_path, dpi=300)
        plt.close(fig.fig)
    except Exception as e:
        log.error(f'Error generating figure.', exc_info=e)
    finally:
        plt.close('all')
        

In [ ]:
%matplotlib qt
stim_duration = 40
time_pre_stim_start = 30
time_post_stim = 90
time_post_stim_start = stim_duration + time_post_stim

sampling_period = 0.5
ref_time_array = np.arange(-1 * time_pre_stim_start, time_post_stim_start, 
                           sampling_period)
ref_time_index = pd.Index(ref_time_array, name=TIME_CN)
log.info(f'{ref_time_array.shape = }')
log.info(f'{ref_time_index = }')

stim_seq_dfs = []
for _, stim_loc_row in stim_log_subset_long_df.iterrows():
    log.info(f'{stim_loc_row[EXPERIMENT_ID_CN] = }, {stim_loc_row[RUN_ID_CN] = }')
    
    activity_traces_subset_df: pd.DataFrame = activity_traces_df.loc[
        (activity_traces_df[EXPERIMENT_ID_CN] == stim_loc_row[EXPERIMENT_ID_CN]) &
        (activity_traces_df[RUN_ID_CN] == stim_loc_row[RUN_ID_CN]), :].copy()
    
    if activity_traces_subset_df.shape[0] == 0:
        continue
    
    activity_traces_subset_df[TIME_CN] = activity_traces_subset_df[TIME_CN] - stim_loc_row[REL_START_TIME_CN]
    activity_traces_subset_df = activity_traces_subset_df.loc[
        (activity_traces_subset_df[TIME_CN] >= (-1 * (2 + time_pre_stim_start))) &
        (activity_traces_subset_df[TIME_CN] <= (2 + time_post_stim_start)), :]
    
    log.info(f'{activity_traces_subset_df.shape = }')
    
    activity_traces_subset_data = activity_traces_subset_df.pivot(
        index=TIME_CN,
        columns=[REGION_CN, LAT_SIDE_CN, EXPERIMENT_ID_CN, RUN_ID_CN],
        values=MEAN_CN)
    log.info(f'{activity_traces_subset_data.shape = }')    
    
    baseline_mean = activity_traces_subset_data.loc[
        (activity_traces_subset_data.index < -1), :].quantile(0.5, axis=0)
    log.info(f'{baseline_mean.shape = }')
    
    activity_traces_subset_data = (activity_traces_subset_data - baseline_mean) / baseline_mean
    interp_res = scipy.interpolate.interp1d(
        activity_traces_subset_data.index,
        activity_traces_subset_data,
        axis=0,
        kind='linear',
        bounds_error=False)(ref_time_array)
    interp_res = scipy.ndimage.gaussian_filter1d(
        interp_res, 0.5 / sampling_period, axis=0, mode='nearest')
    
    log.info(f'{interp_res.shape = }')
    
    activity_traces_subset_data = pd.DataFrame(
        interp_res,
        index=ref_time_index,
        columns=activity_traces_subset_data.columns)       
    
    # make filtered data back into dataframe
    filt_activity_traces_subset_df = (
        activity_traces_subset_data.melt(value_name=MEAN_CN, ignore_index=False)
        .reset_index())
    filt_activity_traces_subset_df[SPATIAL_LAYOUT_CN] = stim_loc_row[SPATIAL_LAYOUT_CN]
    filt_activity_traces_subset_df[STIM_TYPE_CN] = stim_loc_row[STIM_TYPE_CN]
    log.info(f'{filt_activity_traces_subset_df.shape = }')
    
    stim_seq_dfs.append(filt_activity_traces_subset_df)
    
stim_seq_df = pd.concat(stim_seq_dfs, axis=0, ignore_index=True)

stim_seq_df.info()
stim_seq_df

In [ ]:
%matplotlib qt

for i, region_chunk in enumerate(chunks(all_regions, 1)):
    # log.info(f'{region_chunk = }')
    
    save_path = os.path.join(multi_experiment_output_dir, 
                             'figures',
                             f'stimulation-multi-stim-type',
                             f'multi_stim_'
                             f'{stim_duration:.0f}s-pulse_'
                             f'{i:03d}-region_'
                             f'aggregated-results.png')
    
    if os.path.exists(save_path):
        continue
                
    try:
        subset_df = stim_seq_df.loc[stim_seq_df[REGION_CN].isin(region_chunk), :]
        
        fig = sns.relplot(
            data=subset_df,
            x=TIME_CN,
            y=MEAN_CN,
            hue=LAT_SIDE_CN,
            col=SPATIAL_LAYOUT_CN,
            row=STIM_TYPE_CN,        
            kind='line',
            facet_kws={'sharey': True, 'sharex': True},
            errorbar=('se', 2),
            height=2.5,
            aspect=1.5)
    
        (fig
         .map(plt.axvline, x=0, color='k', dashes=(2, 1), linewidth=1, zorder=0)
         .map(plt.axhline, y=0, color='k', dashes=(2, 1), linewidth=1, zorder=0)
         .map(plt.fill_between, x=[0, stim_duration], y1=[-5, -5], y2=[5, 5], 
              color='k', alpha=0.1, linewidth=0, zorder=-1)
         .set_axis_labels('Time (s)', 'Delta F / F (a.u.)')
         .set(xlim=(-1 * time_pre_stim_start, time_post_stim_start),
              ylim=(-0.3, 0.6))
         .tight_layout(h_pad=0.75, w_pad=0.5, rect=(0, 0, 0.93, 0.93)))
        
        fig.fig.suptitle(f'{region_chunk[0]}'
                         f'\nAggregated Results for Stimulation with different Odors, '
                         f'{stim_duration} s Pulse '
                         f'(two standard errors shaded)', 
                         y=0.99)
        
        # sns.despine(offset=2, trim=True)
    
        csutils.touchdir(os.path.dirname(save_path))
        log.info(f'Saving figure to disk...')
        fig.savefig(save_path, dpi=300)
        plt.close(fig.fig)
    except Exception as e:
        log.error(f'Error generating figure.', exc_info=e)
    finally:
        plt.close('all')

In [143]:
selected_stim_types = ['cdv-2mM', 'aa-1.3mM', 'aa-1.3mM&cdv-2mM']
stim_log_subset_df: pd.DataFrame = (
    stimulation_log_df
    .loc[stimulation_log_df[STIM_TYPE_CN].isin(selected_stim_types)
         & stimulation_log_df[DURATION_CN].isin([30.]), :])
stim_log_subset_df.info()
stim_log_subset_df


<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 44 to 248
Data columns (total 11 columns):
 #   Column                   Non-Null Count  Dtype   
---  ------                   --------------  -----   
 0   description              24 non-null     object  
 1   duration_s               24 non-null     float64 
 2   left                     24 non-null     category
 3   middle                   24 non-null     category
 4   right                    24 non-null     category
 5   flag                     24 non-null     bool    
 6   stim-type                24 non-null     category
 7   actual-rel-start-time_s  24 non-null     float64 
 8   experiment-id            24 non-null     category
 9   run-id                   24 non-null     category
 10  stim-step-index          24 non-null     int64   
dtypes: bool(1), category(6), float64(2), int64(1), object(1)
memory usage: 2.9+ KB


,description,duration_s,left,middle,right,flag,stim-type,actual-rel-start-time_s,experiment-id,run-id,stim-step-index
44,cadaverine sustain,30.0,cadaverine (2 mM),fish water,cadaverine (2 mM),False,cdv-2mM,668.007909,cs-ii-33,188,16
46,aa sustain,30.0,amino acid (1.3 mM),fish water,amino acid (1.3 mM),False,aa-1.3mM,818.009439,cs-ii-33,188,18
48,aa sustain,30.0,amino acid (1.3 mM),fish water,amino acid (1.3 mM),False,aa-1.3mM,968.010798,cs-ii-33,188,20
50,cadaverine sustain,30.0,cadaverine (2 mM),fish water,cadaverine (2 mM),False,cdv-2mM,1118.010576,cs-ii-33,188,22
68,cadaverine sustain,30.0,cadaverine (2 mM),fish water,cadaverine (2 mM),False,cdv-2mM,668.003620,cs-ii-33,189,16
70,aa sustain,30.0,amino acid (1.3 mM),fish water,amino acid (1.3 mM),False,aa-1.3mM,818.005731,cs-ii-33,189,18
72,aa sustain,30.0,amino acid (1.3 mM),fish water,amino acid (1.3 mM),False,aa-1.3mM,968.005218,cs-ii-33,189,20
74,cadaverine sustain,30.0,cadaverine (2 mM),fish water,cadaverine (2 mM),False,cdv-2mM,1118.005257,cs-ii-33,189,22
92,cadaverine sustain,30.0,cadaverine (2 mM),fish water,cadaverine (2 mM),False,cdv-2mM,668.003338,cs-ii-33,190,16
94,aa sustain,30.0,amino acid (1.3 mM),fish water,amino acid (1.3 mM),False,aa-1.3mM,818.003908,cs-ii-33,190,18


In [146]:
%matplotlib qt
stim_duration = 30
time_pre_stim_start = 30
time_post_stim = 60
time_post_stim_start = stim_duration + time_post_stim

sampling_period = 0.5
ref_time_array = np.arange(-1 * time_pre_stim_start, time_post_stim_start, 
                           sampling_period)
ref_time_index = pd.Index(ref_time_array, name=TIME_CN)
log.info(f'{ref_time_array.shape = }')
log.info(f'{ref_time_index = }')

stim_seq_dfs = []
for _, stim_loc_row in stim_log_subset_df.iterrows():
    log.info(f'{stim_loc_row[EXPERIMENT_ID_CN] = }, {stim_loc_row[RUN_ID_CN] = }')
    
    activity_traces_subset_df: pd.DataFrame = activity_traces_df.loc[
        (activity_traces_df[EXPERIMENT_ID_CN] == stim_loc_row[EXPERIMENT_ID_CN]) &
        (activity_traces_df[RUN_ID_CN] == stim_loc_row[RUN_ID_CN]), :].copy()
    
    if activity_traces_subset_df.shape[0] == 0:
        continue
    
    activity_traces_subset_df[TIME_CN] = activity_traces_subset_df[TIME_CN] - stim_loc_row[REL_START_TIME_CN]
    activity_traces_subset_df = activity_traces_subset_df.loc[
        (activity_traces_subset_df[TIME_CN] >= (-1 * (2 + time_pre_stim_start))) &
        (activity_traces_subset_df[TIME_CN] <= (2 + time_post_stim_start)), :]
    
    if activity_traces_subset_df.shape[0] == 0:
        continue
    
    log.info(f'{activity_traces_subset_df.shape = }')
    
    activity_traces_subset_data = activity_traces_subset_df.pivot(
        index=TIME_CN,
        columns=[REGION_CN, LAT_SIDE_CN, EXPERIMENT_ID_CN, RUN_ID_CN],
        values=MEAN_CN)
    log.info(f'{activity_traces_subset_data.shape = }')    
    
    baseline_mean = activity_traces_subset_data.loc[
        (activity_traces_subset_data.index < -1), :].quantile(0.5, axis=0)
    log.info(f'{baseline_mean.shape = }')
    
    activity_traces_subset_data = (activity_traces_subset_data - baseline_mean) / baseline_mean
    interp_res = scipy.interpolate.interp1d(
        activity_traces_subset_data.index,
        activity_traces_subset_data,
        axis=0,
        kind='linear',
        bounds_error=False)(ref_time_array)
    interp_res = scipy.ndimage.gaussian_filter1d(
        interp_res, 0.5 / sampling_period, axis=0, mode='nearest')
    
    log.info(f'{interp_res.shape = }')
    
    activity_traces_subset_data = pd.DataFrame(
        interp_res,
        index=ref_time_index,
        columns=activity_traces_subset_data.columns)       
    
    # make filtered data back into dataframe
    filt_activity_traces_subset_df = (
        activity_traces_subset_data.melt(value_name=MEAN_CN, ignore_index=False)
        .reset_index())
    filt_activity_traces_subset_df[STIM_TYPE_CN] = stim_loc_row[STIM_TYPE_CN]
    log.info(f'{filt_activity_traces_subset_df.shape = }')
    
    stim_seq_dfs.append(filt_activity_traces_subset_df)
    
stim_seq_df = pd.concat(stim_seq_dfs, axis=0, ignore_index=True)

stim_seq_df.info()
stim_seq_df

matplotlib.pyplot ( 339): DEBUG : Loaded backend Qt5Agg version 5.15.7.
local-nb_cs-ii-[33,34]_activity_stimulus_overlays (  11): INFO : ref_time_array.shape = (240,)
local-nb_cs-ii-[33,34]_activity_stimulus_overlays (  12): INFO : ref_time_index = Float64Index([-30.0, -29.5, -29.0, -28.5, -28.0, -27.5, -27.0, -26.5, -26.0,
              -25.5,
              ...
               85.0,  85.5,  86.0,  86.5,  87.0,  87.5,  88.0,  88.5,  89.0,
               89.5],
             dtype='float64', name='time', length=240)
local-nb_cs-ii-[33,34]_activity_stimulus_overlays (  16): INFO : stim_loc_row[EXPERIMENT_ID_CN] = 'cs-ii-33', stim_loc_row[RUN_ID_CN] = 188
local-nb_cs-ii-[33,34]_activity_stimulus_overlays (  16): INFO : stim_loc_row[EXPERIMENT_ID_CN] = 'cs-ii-33', stim_loc_row[RUN_ID_CN] = 188
local-nb_cs-ii-[33,34]_activity_stimulus_overlays (  16): INFO : stim_loc_row[EXPERIMENT_ID_CN] = 'cs-ii-33', stim_loc_row[RUN_ID_CN] = 188
local-nb_cs-ii-[33,34]_activity_stimulus_overlays (  16): INF

,time,region,lateral-side,experiment-id,run-id,fluorescence-mean,stim-type
0,-30.0,"Mauthner Cell Axon Cap, Rhombencephalon",left,cs-ii-33,189,-0.142768,cdv-2mM
1,-29.5,"Mauthner Cell Axon Cap, Rhombencephalon",left,cs-ii-33,189,-0.280842,cdv-2mM
2,-29.0,"Mauthner Cell Axon Cap, Rhombencephalon",left,cs-ii-33,189,-0.408969,cdv-2mM
3,-28.5,"Mauthner Cell Axon Cap, Rhombencephalon",left,cs-ii-33,189,-0.333266,cdv-2mM
4,-28.0,"Mauthner Cell Axon Cap, Rhombencephalon",left,cs-ii-33,189,0.014721,cdv-2mM
...,...,...,...,...,...,...,...
1547035,87.5,Rhombencephalon,right,cs-ii-34,209,-0.090557,aa-1.3mM
1547036,88.0,Rhombencephalon,right,cs-ii-34,209,-0.087677,aa-1.3mM
1547037,88.5,Rhombencephalon,right,cs-ii-34,209,-0.086163,aa-1.3mM
1547038,89.0,Rhombencephalon,right,cs-ii-34,209,-0.085158,aa-1.3mM


In [151]:
%matplotlib qt

for i, region_chunk in enumerate(chunks(all_regions, 5)):
    # log.info(f'{region_chunk = }')
    
    save_path = os.path.join(multi_experiment_output_dir, 
                             'figures',
                             f'stimulation-aa-cad',
                             f'multi_stim_'
                             f'{stim_duration:.0f}s-pulse_'
                             f'{i:03d}-5-region-set_'
                             f'aggregated-results.png')
    csutils.touchdir(os.path.dirname(save_path))
    
    if os.path.exists(save_path):
        continue
                
    try:
        subset_df = stim_seq_df.loc[stim_seq_df[REGION_CN].isin(region_chunk), :]
        
        fig = sns.relplot(
            data=subset_df,
            x=TIME_CN,
            y=MEAN_CN,
            hue=LAT_SIDE_CN,
            col=REGION_CN,
            row=STIM_TYPE_CN,        
            kind='line',
            facet_kws={'sharey': True, 'sharex': True},
            errorbar=('se', 2),
            height=2.5,
            aspect=1.5)
    
        (fig
         .map(plt.axvline, x=0, color='k', dashes=(2, 1), linewidth=1, zorder=0)
         .map(plt.axhline, y=0, color='k', dashes=(2, 1), linewidth=1, zorder=0)
         .map(plt.fill_between, x=[0, stim_duration], y1=[-5, -5], y2=[5, 5], 
              color='k', alpha=0.1, linewidth=0, zorder=-1)
         .set_titles('region = {col_name}\nstimulation = {row_name}')
         .set_axis_labels('Time (s)', 'Delta F / F (a.u.)')
         .set(xlim=(-1 * time_pre_stim_start, time_post_stim_start),
              ylim=(-0.3, 0.6))
         .tight_layout(h_pad=0.75, w_pad=0.5, rect=(0, 0, 0.93, 0.93)))
        
        fig.fig.suptitle(f'\nAggregated Results for Stimulation with different Odors, '
                         f'{stim_duration} s Pulse '
                         f'(two standard errors shaded)', 
                         y=0.99)
        
        # sns.despine(offset=2, trim=True)
        
        log.info(f'Saving figure to disk...')
        fig.savefig(save_path, dpi=300)
        
        plt.close(fig.fig)
    except Exception as e:
        log.error(f'Error generating figure.', exc_info=e)
    finally:
        plt.close('all')

matplotlib.pyplot ( 339): DEBUG : Loaded backend Qt5Agg version 5.15.7.


I:\CorbanSwain\repos\zmia\envs\zmia-env-3.9\lib\site-packages\seaborn\axisgrid.py:118: UserWarning:

The figure layout has changed to tight

I:\CorbanSwain\repos\zmia\envs\zmia-env-3.9\lib\site-packages\seaborn\axisgrid.py:118: UserWarning:

The figure layout has changed to tight

I:\CorbanSwain\repos\zmia\envs\zmia-env-3.9\lib\site-packages\seaborn\axisgrid.py:118: UserWarning:

The figure layout has changed to tight

I:\CorbanSwain\repos\zmia\envs\zmia-env-3.9\lib\site-packages\seaborn\axisgrid.py:118: UserWarning:

The figure layout has changed to tight



local-nb_cs-ii-[33,34]_activity_stimulus_overlays (  52): INFO : Saving figure to disk...


I:\CorbanSwain\repos\zmia\envs\zmia-env-3.9\lib\site-packages\seaborn\axisgrid.py:118: UserWarning:

The figure layout has changed to tight

I:\CorbanSwain\repos\zmia\envs\zmia-env-3.9\lib\site-packages\seaborn\axisgrid.py:118: UserWarning:

The figure layout has changed to tight

I:\CorbanSwain\repos\zmia\envs\zmia-env-3.9\lib\site-packages\seaborn\axisgrid.py:118: UserWarning:

The figure layout has changed to tight

I:\CorbanSwain\repos\zmia\envs\zmia-env-3.9\lib\site-packages\seaborn\axisgrid.py:118: UserWarning:

The figure layout has changed to tight



local-nb_cs-ii-[33,34]_activity_stimulus_overlays (  52): INFO : Saving figure to disk...


...

I:\CorbanSwain\repos\zmia\envs\zmia-env-3.9\lib\site-packages\seaborn\axisgrid.py:118: UserWarning:

The figure layout has changed to tight

I:\CorbanSwain\repos\zmia\envs\zmia-env-3.9\lib\site-packages\seaborn\axisgrid.py:118: UserWarning:

The figure layout has changed to tight

I:\CorbanSwain\repos\zmia\envs\zmia-env-3.9\lib\site-packages\seaborn\axisgrid.py:118: UserWarning:

The figure layout has changed to tight

I:\CorbanSwain\repos\zmia\envs\zmia-env-3.9\lib\site-packages\seaborn\axisgrid.py:118: UserWarning:

The figure layout has changed to tight



local-nb_cs-ii-[33,34]_activity_stimulus_overlays (  52): INFO : Saving figure to disk...


local-nb_cs-ii-[33,34]_activity_stimulus_overlays (  57): ERROR : Error generating figure.
Traceback (most recent call last):
  File "I:\CorbanSwain\repos\zmia\envs\zmia-env-3.9\lib\site-packages\pandas\core\indexes\base.py", line 3802, in get_loc
    return self._engine.get_loc(casted_key)
  File "pandas\_libs\index.pyx", line 138, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\index.pyx", line 165, in pandas._libs.index.IndexEngine.get_loc
  File "pandas\_libs\hashtable_class_helper.pxi", line 5745, in pandas._libs.hashtable.PyObjectHashTable.get_item
  File "pandas\_libs\hashtable_class_helper.pxi", line 5753, in pandas._libs.hashtable.PyObjectHashTable.get_item
KeyError: 'y'

The above exception was the direct cause of the following exception:

Traceback (most recent call last):
  File "C:\Users\CorbanSwain\AppData\Local\Temp\ipykernel_21896\2326515018.py", line 21, in <module>
    fig = sns.relplot(
  File "I:\CorbanSwain\repos\zmia\envs\zmia-env-3.9\lib\site-packa

local-nb_cs-ii-[33,34]_activity_stimulus_overlays (  52): INFO : Saving figure to disk...


RuntimeError: generator raised StopIteration